## Class Announcements

**Due Friday**:
- D3
- Project Proposal (Template in repo, turn in via repo)
- Weekly project survey check in (optional, released Wednesdays!) on Canvas

**Notes**:
- Repo invites: missed yours? Ask on Piazza in private message to instructors!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/COGS108/Lectures-Sp22/blob/master/04_data/04_02_descriptive.ipynb)

# Descriptive Analysis

- Size
- Missingness
- Shape
- Central Tendency
- Variability

<div class="alert alert-success">
The goal of a <b>descriptive analysis</b> is to understand and summarize information about the variables stored in your dataset.
</div>

### Setup

The packages and settings we'll use in this workbook:

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (17, 7) #increase figure size

import seaborn as sns
sns.set(style='white', font_scale=2) #set style

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import uniform, norm, bernoulli, poisson

#improve resolution
#comment this line if erroring on your machine/screen
%config InlineBackend.figure_format ='retina'

## The Data

To walk through these concepts today, we're going to use your responses from after the Data Intuition Lecture.

In [ ]:
# read data into Python
df = pd.read_csv('https://raw.githubusercontent.com/COGS108/Lectures-Sp22/master/04_data/data/fermi_wi22.csv')

In [ ]:
# take a look at the data
df[:35]

### Data Cleaning & Wrangling

Tidy Data Rules (Review):
1. Every observation in a row
2. Every variable in a column
3. If multiple tables, column on which to merge

https://forms.gle/iW1HwXXqrRnvdkAw7

<img src="https://raw.githubusercontent.com/COGS108/Lectures-Sp22/master/04_data/images/qr-code(15).png" width=600>

#### Clicker Question #1

Are these data in the tidy data format?

- A) Yes, these data are ready to analyze
- B) Yes, but there is more work to do before analysis
- C) No, not tidy
- D) Have no idea what you're talking about

**Brainstorming**

What should we do before we use these data?



hmm...

lets make a list


- ideas from you
- more ideas


- shorten column/variable names
- standardize responses
    - make measurements uniform
    - remove units
- string to int/float (want to work with #s)
- consider missing values

In [ ]:
# change column names
df.columns = ['timestamp', 'hair_growth', 'crammed', 'SAN_NYC']
df.head()

In [ ]:
# check type of each Series (column)
df.dtypes

In [ ]:
df['timestamp'] = pd.to_datetime( df['timestamp'])

In [ ]:
df.dtypes

## Size

As discussed previously, knowing and checking the size of your data helps you:
- understand what information you have
- know if it read into Python correctly
- determine what analyses are appropriate

In [ ]:
# determine rows and columns in df
df.shape

We now know that we have information about 85 students across 4 variables.

## Missingness

Data can be missing for all kinds of reasons. It's your job to determine if:
- values are missing at random
- values are missing due to data entry errors
- values are missing due to faulty data collection



In [ ]:
df.isnull().all(axis=1)

In [ ]:
# True if row contains at least one null value
# axis argument: 0 for reducing by ‘index’, 1 for reducing by ‘columns’,
null_rows = df.isnull().any(axis='columns')
df[null_rows].shape

In [ ]:
# columns with missing values
df.columns[df.isnull().any(axis='rows')]

In [ ]:
# number of missing values by column
df.isnull().sum()

## Cleaning: Hair Growth

How fast does human hair grow (cm/yr)?

In [ ]:
# take a look at unique values
df["hair_growth"].unique()

In [ ]:
# standardize height column
def standardize_hair(string):
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()    

    # take care of inclded unit cases   
    string = string.replace("cm a year", "")
    string = string.replace("cm/year", "")
    string = string.replace("cm/ye", "")
    string = string.replace("centimeters", "")
    string = string.replace("cm per year", "")
    string = string.replace("cm/yr", "")
    string = string.replace("cm/1yr", "")
    string = string.replace("cm/y", "")
    string = string.replace("cm/1 year", "")
    string = string.replace("year", "")
    string = string.replace("yr", "")
    string = string.replace("cm", "")    
    string = string.replace("/", "")
    string = string.replace(",", "")
    string = string.replace("eh, I'm  bald", "")
    string = string.replace('cm approx.', "")
    string = string.replace("cm/yr, cuz it grows 2cm in a month", "")
    string = string.replace("^2", "0")
    
    string = string.strip()

    # convert to numeric
    try:
        output = float(eval(string))
    except:
        output = np.nan
    
    return output

In [ ]:
# apply function across values in hair growth columns
df["hair_growth"] = df["hair_growth"].apply(standardize_hair)
df["hair_growth"].unique()

## Cleaning: Crammed

If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?

In [ ]:
df['crammed'].unique()

In [ ]:
# standardize crammed column
def standardize_crammed(string):    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of commas
    string = string.replace(",", "")

    # take care of specific cases
    string = string.replace("texas?", "696200")
    string = string.replace("i'm bad at area lol", "NaN")
    string = string.replace("1.5 billion ft^2 to whatever that is in km^2", "139.35456")
    string = string.replace("the whole earth","509600000")
    
    # take care of inclded unit cases
    string = string.replace("^2", "")
    string = string.replace("²", "")
    string = string.replace("km", "")
    
    # take care of scientific notation / word cases
    string = string.replace(" million", "000000")
    string = string.replace(" billion", "000000000")

    string = string.strip()


    # convert to numeric
    try:
        output = float(eval(string))
    except:
        output = np.nan
    
    return output

In [ ]:
# apply function across values in crammed columns
df["crammed"] = df["crammed"].apply(standardize_crammed)
df["crammed"].unique()

## Cleaning: San Diego to NYC

How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

In [ ]:
df['SAN_NYC'].unique()

In [ ]:
# standardize distance column
def standardize_distance(string):
    
    orig = string
    output = None
    
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of commas
    string = string.replace(",", "")
    
    # remove units
    string = string.replace("days", "")
    
    # remove uncertainty
    string = string.replace("?", "")

    # remove modifier
    string = string.replace("over", "")

    # take care of non-days answers
    string = string.replace("2 months", "60")
    string = string.replace("4 weeks", "30")

    # take care of scientific notation 
    string = string.replace("10^3", "10000")
    
    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [ ]:
# alternate approach to the same task
def dist_helper(distance):
    if len(distance) == 1:
        return distance[0]
    if distance[1] == '*':
        return str(float(distance[0]) * float(distance[2]))
    if distance[1] == '/':
        return str(float(distance[0]) / float(distance[2]))

df["SAN_NYC_alt"] = pd.to_numeric(df["SAN_NYC"].
                              str.lower().
                              replace(",", "").
                              replace({"days?":"", 
                                       "\^":"e", "^\([\w */)=?]+$":"1000", 
                                       " months?":" * 30", 
                                       " hours?": " / 24", 
                                       " years?": " * 365"}, regex = True).
                              str.strip().
                              str.split(' ').
                              apply(dist_helper), errors = 'coerce')


# OOOPS!
I left this as it was to correct a previous quarter's outputs.  We don't have the correct wrangling here for the current quarter.

So let's leave this one as an exercise for YOU!  How do you want to handle wrangling this, dealing with all the possible issues, and making certain that each issue is handled!

In [ ]:
# insert your code here!

## Shape

The shape of your data dictates what analyses you can do. Today, we'll review a number of different distributions (shapes) data can take and examples of data that take that distribution.

### Uniform Distribution

<div class="alert alert-success">
The Uniform distribution has the property that every outcome has the equal probability of occurring. In other words, all outcomes are equally likely.
</div>

In [ ]:
dat = uniform.rvs(size=10000)
sns.histplot(dat, bins=20);

The **probability of rolling a given number on a fair die** is the same each time you roll the die - an example of a Uniform distribution.

The **probability of pulling a spade out of a deck of cards** is the same each time you pull a card out of the deck.

The **probability of flipping a heads each time you flip a fair coin** is the same each time you flip the coin.

### Normal Distribution

<div class="alert alert-success">
The Normal (also Gaussian, or 'Bell Curve') distribution, is a distribution defined by its mean and standard deviation.
</div>

In [ ]:
# loc specifies mean
# scale specifies the standard deviation
dat = norm.rvs(loc=0, scale=1, size=10000)
sns.histplot(dat, bins=20);

With a standard Normal curve:
- 68% of the values fall within one standard deviation [-1,1]
- 95% of the values fall within two standard deviations [-2,2]

The **Normal distribution** is taught all over the place, and this is because it shows up all over the place. It's found in nature and across measurements we take all the time. It's also easy to understand and to work with statistically.

The **average height of players in the NBA** follows a Normal distribution, with the average height being 6'7". 

If you were to **flip a fair coin 16 times** and count the number of heads each time...and then repeat this 1000 times, recording the number of heads each time, this would follow a Normal distribution. 8 would be the most popular number of heads, but there would be a normal distribution centered on 8 for these data.

### Bimodal Distributions

<div class="alert alert-success">
A Bimodal Distribution is a distribution with two peaks - it often indicates that you have information about two groups. 
</div>

In [ ]:
loc1, scale1, size1 = (90, 5, 175)
loc2, scale2, size2 = (70, 5, 175)
bi = np.concatenate([np.random.normal(loc=loc1, scale=scale1, size=size1), 
                     np.random.normal(loc=loc2, scale=scale2, size=size2)])
sns.histplot(bi, bins=20);

When **test scores in a class** are bimodal, often one peak describes those students who studied, while the other are those who didn't study or are struggling more with the course material.

Another example of a bimodal distribution are **the number of visitors at a restaurant over time**. Often restaurants will get a peak of visitors at lunchtime and dinnertime, with lulls in between.

### Bernoulli Distribution

<div class="alert alert-success">
A Bernouilli Distribution is a binary distribution - it takes only two values (0 or 1), with some probability $p$. 
</div>

In [ ]:
r = bernoulli.rvs(0.1 , size=10000)
sns.countplot(r, color='#7995C3');

Usually the value 1 indicates 'success' and 0 indicates 'failure'

Whether a **team will win a championship or not** follows a Bernoulli distribution - the team will either win (1 = success) or lose (0 = failure), and there is some probability ($p$) assigned to each of those values.

Similarly, **whether you pass each exam at UCSD** follows a Bernoulli distribution - either you pass (1 = success) or you fail (0 = failure), and there is some probability assinged to each.

There are distributions that are built off of the Bernoulli Distribution, defined as follows:
- **Binomial Distribution**: Number of success in $n$ trials
- **Geometric Distribution**: Number of failures before the first success
- **Negative Binomial Distribution**: Number of failures before the $x^{th}$ success

### Poisson Distribution

<div class="alert alert-success">
The Poisson Distribution models events in fixed intervals of time, given a known average rate (and independent occurences).
</div>

In [ ]:
dat = poisson.rvs(mu=1, size=100000)
sns.histplot(dat);

The **number of visitors a fast food drive-through gets each minute** follows a Poisson distribution. In this case, maybe the average is 3, but there's some variability around that number. 

A Poisson distribution can help calculate the probability of various events related to customers going through the drive-through at a restaurant. It will predict lulls (0 customers) and flurry of activity (5+ customers), allowing staff to plan and schedule more precisely.

https://forms.gle/S6TZ6sE2XVmGuymZ9

<img src="https://raw.githubusercontent.com/COGS108/Lectures-Sp22/master/04_data/images/qr-code(16).png" width=800>

#### Clicker Question #2

Which of the following would you expect to be **bimodal**?

- A) heights from a random sample of females in the US
- B) daily chance of winning the lottery
- C) number of siblings everyone in this class has
- D) distribution of speed limits in the US
- E) ages of everyone in this class

#### Clicker Question #3

The "winning" number in a lottery follows a...

- A) Normal Distribution
- B) Uniform Distribution
- C) Skewed-right distribution
- D) Bimodal Distribtuion
- E) Bernoulli Distribution

#### Shape: Fermi Data

In [ ]:
sns.histplot(df['hair_growth'], kde=False, bins=20);

In [ ]:
sns.histplot(df['crammed'], kde=False, bins=200);

In [ ]:
df.crammed

In [ ]:
sns.histplot(df.query("SAN_NYC < 1000")['SAN_NYC'], kde=False, bins=20);

#.query("SAN_NYC < 10000")

### Data Transformations

We haven't discussed skewed distributions yet; however, all three variables here are **skewed right**, meaning there is a tail off to the right and most values are found near the lower portion of the distribution.

These sorts of distributions are hard to analyze because deviations from the norm are driving the variation in the distribution...even though they are few in number.

Often, when you have skwewed data, you'll want to transform the variable.

In [ ]:
# log transformed data
sns.histplot(np.log10(df['hair_growth'][df['hair_growth'].notnull()]), bins=10)
plt.xlabel('Hair Growth');

After transforming the data, the values appear approximately Normal. Those high values are no longer driving variation in the data. However, we're now on a log-scale, which sometimes makes interpretation a bit more difficult.

### Outliers

Outliers are values that fall outside the typical range of your dataset. These can occur for all types of reasons:

- data entry errors
- poor sampling procedures
- technical or mechanical errors
- unexpected changes in weather
- extreme values
- people giving incorrect information
- etc.

<div class="alert alert-danger">
Caution: Observations should only be removed from your dataset if you have a valid reason to do so. If you remove outliers from your dataset, your report should be <b>very</b> clear that you did so.
</div>

![outliers](https://raw.githubusercontent.com/COGS108/Lectures-Sp22/master/04_data/images/outliers.png)

Link to Tweet: https://twitter.com/ChelseaParlett/status/1356285012375556109  
Thread on possible approaches: https://twitter.com/IsabellaGhement/status/1356645319799308288

## Central Tendency

- mean 
- median
- mode

The Central Tendency tells you the 'typical' value for an observation in your dataset.

### Mean


$$ \bar x = \frac{\sum\limits_{i = 1}^N x_i}N  $$


- $x_i$ = ith element of the sample
- $\bar x$ = sample mean
- $N$ = sample size


In [ ]:
# to calculate mean
sum(df['hair_growth'])/len(df)

In [ ]:
df['hair_growth'].mean()

In [ ]:
# check mean for each column
df.mean()

### Median

In [ ]:
# check median for each column
df.median()

### Median vs. Mean

When the median and mean are not similar to one another...what's the best approach?

In [ ]:
# relook at the distribution for bodywt
sns.histplot(df['hair_growth'], bins=10);

In [ ]:
# median and mean for same series
print( 'median: ', df['hair_growth'].median())
print( 'mean: ', df['hair_growth'].mean())

In [ ]:
# take a look at it all together
ax = sns.histplot(df['hair_growth'], bins=10);
ax.axvline(df['hair_growth'].mean(), color='darkred', linestyle='--', label='mean');
ax.axvline(df['hair_growth'].median(), color='#2e2e2e', linestyle='--', label='median')
ax.legend();

#### Clicker Question #4

Which of the following is the best way to measure the central tendency of `hair_growth` in these data?

- A) mean
- B) median
- C) mode

In [ ]:
# increase the number of bins here
ax = sns.histplot(df['hair_growth'], bins=100);
ax.axvline(df['hair_growth'].mean(), color='darkred', linestyle='--', label='mean');
ax.axvline(df['hair_growth'].median(), color='#2e2e2e', linestyle='--', label='median')
ax.legend();

#### How did y'all do?

In [ ]:
# compare to actual value: 15 cm/year (~6 in)
df["hair_growth"].median()

In [ ]:
# compare to actual value: 1,000-10,000 km^2)
df['crammed'].median()

In [ ]:
# compare to actual value: 38 days)
df['SAN_NYC'].median()

Calculating the mean and median of your sample is helpful when dealing with **quantitative variables**.

When working with **categorical variables**, knowing the mode is helpful.

### Mode

When working with categorical data, the mode is the most common value in the dataset.

## Variability

- Range
- IQR
- Variance & Standard Deviation

### Range

The highest value minus the lowest value.

In [ ]:
# determine the 25th and 75th percentiles
min_val = df['hair_growth'].min()
max_val = df['hair_growth'].max()
range_vals =  max_val - min_val
print(max_val, '-' , min_val,' = ',  range_vals)

### IQR (Interquartile Range)

75th percentile - 25th percentile

In [ ]:
# determine the 25th and 75th percentiles
lower, upper = np.percentile(df['hair_growth'], [25, 75])
lower, upper

In [ ]:
# calculate IQR
iqr = upper - lower
iqr

In [ ]:
df['hair_growth'].quantile([0.25,0.75])

In [ ]:
# visualizing IQR
sns.boxplot(x='hair_growth', data=df);

### Variance & Standard Deviation

- variance
    - measures how close the values in the distribution are to the middle of the distribution
    - average squared difference of the scores from  the mean
- standard deviation
    - square root of the variance

#### Variance

$$ s^2 = \frac {\sum\limits_{i = 1}^N {\left( {x_i - \bar x} \right)^2 }} {(N-1)} $$

- $s^2$ = sample variance
- $x_i$ = ith element of the sample
- $\bar x$ = mean of the sample
- $N$ = sample size


In [ ]:
# the math behind sample variance
var = sum((df['hair_growth'] - df['hair_growth'].mean()) ** 2 )/(len(df) - 1) 
var

In [ ]:
# calculate variance using pandas
var = df['hair_growth'].var()
var

#### Standard Deviation
square root of the variance

$$ s = \sqrt {\frac {\sum\limits_{i = 1}^N {\left( {x_i - \bar x} \right)^2 }} {(N-1)}} $$

In [ ]:
np.sqrt(var)

In [ ]:
# calculate variance using pandas
sd = df['hair_growth'].std()
sd 

## Descriptive Tables

![descriptive table](https://raw.githubusercontent.com/COGS108/Lectures-Sp22/master/04_data/images/descriptive.png)

### Why Central Tendency Doesn't Tell the Whole Story

In [ ]:
# generate two different normal distributions
dist_1 = np.random.normal(5, 2, 1000)
dist_2 = np.random.normal(5, 10, 1000)

In [ ]:
# plot distributions side by side
sns.kdeplot(dist_1, label="mean: 5, sd: 2")
sns.kdeplot(dist_2, label="mean: 5, sd: 10")
plt.legend();

## Anscombe's Quartet: A Cautionary Tale

Code in this example taken from [here](https://matplotlib.org/gallery/specialty_plots/anscombe.html).

In [ ]:
x = np.array([10, 8, 13, 9, 11, 14, 6, 4, 12, 7, 5])
y1 = np.array([8.04, 6.95, 7.58, 8.81, 8.33, 9.96, 7.24, 4.26, 10.84, 4.82, 5.68])
y2 = np.array([9.14, 8.14, 8.74, 8.77, 9.26, 8.10, 6.13, 3.10, 9.13, 7.26, 4.74])
y3 = np.array([7.46, 6.77, 12.74, 7.11, 7.81, 8.84, 6.08, 5.39, 8.15, 6.42, 5.73])
x4 = np.array([8, 8, 8, 8, 8, 8, 8, 19, 8, 8, 8])
y4 = np.array([6.58, 5.76, 7.71, 8.84, 8.47, 7.04, 5.25, 12.50, 5.56, 7.91, 6.89])

In [ ]:
def fit(x):
    return 3 + 0.5 * x

xfit = np.array([np.min(x), np.max(x)])

plt.subplot(221)
plt.plot(x, y1, 'ks', xfit, fit(xfit), 'r-', lw=2)
plt.axis([2, 20, 2, 14])
plt.setp(plt.gca(), xticklabels=[], yticks=(4, 8, 12), xticks=(0, 10, 20))
plt.text(3, 12, 'I', fontsize=20)

plt.subplot(222)
plt.plot(x, y2, 'ks', xfit, fit(xfit), 'r-', lw=2)
plt.axis([2, 20, 2, 14])
plt.setp(plt.gca(), xticks=(0, 10, 20), xticklabels=[],
         yticks=(4, 8, 12), yticklabels=[], )
plt.text(3, 12, 'II', fontsize=20)

plt.subplot(223)
plt.plot(x, y3, 'ks', xfit, fit(xfit), 'r-', lw=2)
plt.axis([2, 20, 2, 14])
plt.text(3, 12, 'III', fontsize=20)
plt.setp(plt.gca(), yticks=(4, 8, 12), xticks=(0, 10, 20))

plt.subplot(224)
xfit = np.array([np.min(x4), np.max(x4)])
plt.plot(x4, y4, 'ks', xfit, fit(xfit), 'r-', lw=2)
plt.axis([2, 20, 2, 14])
plt.setp(plt.gca(), yticklabels=[], yticks=(4, 8, 12), xticks=(0, 10, 20))
plt.text(3, 12, 'IV', fontsize=20)

# verify the stats
pairs = (x, y1), (x, y2), (x, y3), (x4, y4)
for x, y in pairs:
    print('mean=%1.2f, std=%1.2f, r=%1.2f' % (np.mean(y), np.std(y),
          np.corrcoef(x, y)[0][1]))

plt.show()

### Draw the Graph (What EDA is all about!)